# "The Eye Of The Typer" Dataset

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import re
from pathlib import Path
from typing import Literal, Iterator
from datetime import datetime, timedelta
from pprint import pprint

import polars as pl
import rerun as rr
import cv2 as cv

from eott_dataset import *
from eott_dataset.rerun import *

In [ ]:
# %env set EOTT_DATASET_PATH
df = read_participant_characteristics()
df

In [ ]:
participants = [Participant.create(**row) for row in df.iter_rows(named=True)]

print("Participants:", len(participants), end="\n\n")

In [ ]:
p = participants[0]
pprint(p)

In [ ]:
rr.init("EOTT", recording_id=p.participant_id, spawn=True)
rr.log(
    "participant",
    rr.TextDocument(
        "\n".join(f"[{key}]\n{value}\n" for key, value in p.to_dict().items())
    ),
    timeless=True,
)

screen_cap: cv.VideoCapture | None = None
webcam_cap: cv.VideoCapture | None = None
webcam_path: Path | None = None

screen_width = p.display_width
screen_height = p.display_height

screen_scale_factor = 1
if p.screen_recording is not None:
    screen_cap = cv.VideoCapture(str(p.screen_recording_path))
    screen_scale_factor = 2
    screen_width = screen_width // screen_scale_factor
    screen_height = screen_height // screen_scale_factor

rr.log(
    f"screen",
    rr.Boxes2D(
        sizes=[[screen_width, screen_height]],
        centers=[[screen_width / 2, screen_height / 2]],
    ),
    timeless=True,
)

rr.log(
    "participant/pupil/left/tobii",
    rr.SeriesLine(name="left pupil diameter (tobii)", color=(255, 255, 0), width=1),
    timeless=True,
)
rr.log(
    "participant/pupil/right/tobii",
    rr.SeriesLine(name="right pupil diameter (tobii)", color=(255, 0, 255), width=1),
    timeless=True,
)

timeline_df = get_timeline(p)

frame_index: int
webcam_index: int | None
study_name: str | None
offset_time: timedelta
source_name: Source
for log_index, (
    frame_index,
    webcam_index,
    study_name,
    source_name,
    offset_time,
) in enumerate(timeline_df.iter_rows()):
    study_name = Study(study_name) if study_name is not None else None

    rr.set_time_sequence("log_index", log_index)
    rr.set_time_seconds("capture_time", offset_time.total_seconds())

    timeline_name = f"webcam-{webcam_index}" if source_name == "webcam" else source_name
    rr.set_time_sequence(f"{timeline_name}_index", frame_index)

    match source_name:
        case "tobii":
            rerun_log_tobii(p.tobii_gaze_predictions.row(frame_index, named=True))

        case "screen" if screen_cap is not None:
            assert p.screen_recording is not None
            assert p.screen_offset is not None
            screen_cap = rerun_log_screen(screen_cap)

        case "webcam" if webcam_index is not None and study_name is not None:
            paths = p.get_webcam_video_paths(study=study_name, index=webcam_index)

            if len(paths) > 0 and webcam_path != paths[0]:
                webcam_path = paths[0]
                webcam_cap = cv.VideoCapture(str(webcam_path.absolute()))

            rr.set_time_seconds(
                f"{timeline_name}_time", webcam_cap.get(cv.CAP_PROP_POS_MSEC) / 1_000
            )

            success, image = webcam_cap.read()

            if not success:
                webcam_cap = webcam_cap.release()
                webcam_path = None
                continue

            h, w, _ = image.shape
            image = cv.resize(image, (w // 2, h // 2))
            image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
            rr.log("webcam", rr.Image(image))

        case "log":
            rerun_log_user(p.user_interaction_logs.row(frame_index, named=True))

# end of logging
if screen_cap is not None:
    screen_cap.release()

if webcam_cap is not None:
    webcam_cap.release()